In [1]:
import pandas as pd

from core import Config

config = Config()

HISTORIC_DTYPES_PATH = config.eda_filtered_dir / "eda_filtered_historic_dtypes.csv"
STATIC_DTYPES_PATH = config.eda_filtered_dir / "eda_filtered_static_dtypes.csv"

def _read_dtype_map(path) -> dict[str, str]:
    dtypes_df: pd.DataFrame = pd.read_csv(path, index_col=1)
    return dtypes_df.iloc[:, 1].to_dict()

historic_dtypes: dict[str, str] = _read_dtype_map(HISTORIC_DTYPES_PATH)
static_dtypes: dict[str, str] = _read_dtype_map(STATIC_DTYPES_PATH)

historic_df: pd.DataFrame = pd.read_csv(
    config.median_historic,
    index_col=[0, 1],
    dtype=historic_dtypes
)
static_df: pd.DataFrame = pd.read_csv(
    config.median_static,
    index_col=0,
    dtype=static_dtypes
)

instrument_index = historic_df.index.get_level_values(0)
static_aligned = static_df.reindex(instrument_index)
static_aligned.index = historic_df.index

all_data: pd.DataFrame = pd.concat([historic_df, static_aligned], axis=1)

del historic_df, static_aligned, static_df, instrument_index, static_dtypes, historic_dtypes, HISTORIC_DTYPES_PATH, STATIC_DTYPES_PATH
import gc
gc.collect()

In [ ]:
all_data.to_csv(config.dataset_dir / 'median_all_data.csv')

In [ ]:
import pandas as pd
from core import Config
config = Config()
all_data = pd.read_csv(config.dataset_dir / 'median_all_data.csv', index_col=[0, 1])

In [3]:
training_data: pd.DataFrame = all_data.reset_index()
y: pd.DataFrame = training_data['TR.UpstreamScope3PurchasedGoodsAndServices'].to_frame()
X: pd.DataFrame = training_data.drop('TR.UpstreamScope3PurchasedGoodsAndServices', axis=1)